# **Análise Exploratória de Dados de Logística - Loggi**
Este projeto foi elaborado por meio de uma atividade na EBAC.<br> 

Aluno: [Tobias Melo](https://www.linkedin.com/in/tobias-melow/)<br>
Professor: [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Sumário:**

<ol type="1">
  <li>Contexto;</li>
  <li>Pacotes e bibliotecas;</li>
  <li>Exploração de dados;</li>
  <li>Manipulação;</li>
  <li>Visualização;</li>
  <li>Agradecimentos.</li>
</ol>


---

# **Análise Exploratória de Dados de Logística**

## **1\. Contexto**

#### **1.1. Um pouco sobre a história da Loggi**

A Loggi é uma startup unicórnio brasileira de tecnologia focada em logística. A Loggi começou entregando apenas documentos entre 2013 e 2014. Dois anos depois, entrou no segmento de e-commerce. E, desde 2017, tem atuado nas entregas de alimentos também.


#### **Sobre os dados que trabalharei**

O Loggi Benchmark for Urban Deliveries (BUD) é um repositório do GitHub ([link](https://github.com/loggi/loggibud)) com dados e códigos para problemas típicos que empresas de logística enfrentam: otimização das rotas de entrega, alocação de entregas nos veículos da frota com capacidade limitada, etc. Os dados são sintetizados de fontes públicas (IBGE, IPEA, etc.) e são representativos dos desafios que a startup enfrenta no dia a dia, especialmente com relação a sua escala.

#### **1.2. Dados**
Vou trabalhar com um sub conjunto dos dados originais presentes neste [link](https://github.com/loggi/loggibud/blob/master/docs/quickstart.md). Em especial, o arquivo JSON é consolidado em instâncias de treino de cvrp da cidade de Brasília.

## **2\. Pacotes e bibliotecas**

In [ ]:
# Antes de tudo, vamos instalar o pacote 'geopandas' que não é pré-instalado no Google Colab (programa onde o projeto foi desenvolvido).

!pip install geopandas

In [ ]:
# - 1º pacotes nativos do python;
import json
import os

# - 2º pacotes de terceiros;
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

## **3\. Exploração de dados**

**Tópicos:**

**1º** Coleta de dados;

**2º** Wrangling da estrutura;

**3º** Exploração do schema;

**4º** Dados faltantes.

In [ ]:
# 1º Coleta de dados;

!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

print(f'Tamanho: {len(data)}')

---

*Entendendo como é o arquivo, exemplo:*

```json
[
  {
    "name": "cvrp-0-df-0",
    "region": "df-0",
    "origin": {"lng": -47.802664728268745, "lat": -15.657013854445248},
    "vehicle_capacity": 180,
    "deliveries": [
      {
        "id": "ed0993f8cc70d998342f38ee827176dc",
        "point": {"lng": -47.7496622016347, "lat": -15.65879313293694},
        "size": 10
      },
      {
        "id": "c7220154adc7a3def8f0b2b8a42677a9",
        "point": {"lng": -47.75887552060412, "lat": -15.651440380492554},
        "size": 10
      },
      ...
    ]
  }
]
...


### **3.1. Analisando o arquivo via código**

In [ ]:
# Armazenando a primeira linha do JSON
example = data[0]

# Chaves do JSON
print(f'Chaves: {example.keys()}')

### **3.2. Wrangling** 

In [ ]:
# Transformando o arquivo JSON em uma DATAFRAME
deliveries_df = pd.DataFrame(data)

# Lendo as 5 primeiras linhas
deliveries_df.head()

##### **3.2.1. Problema resolvido**

Como observado na leitura das colunas acima, a "origin" vem como um dicionário, para transformar esse dicionário em 2 colunas utilizamos o achatamento dos valores, vamos ao código abaixo.

In [ ]:
"""
  Verificando as 2 colunas com problema

"""

deliveries_df.iloc[[0, 1], [2, 4]]

In [ ]:
""" 
  O pandas tem um método que faz o processo de transformação das chaves do dict p/ colunas & valores

"""
hub_origin_df = pd.json_normalize(deliveries_df["origin"])

# Lendo as 5 primeiras linhas transformadas
hub_origin_df.head()

In [ ]:
"""
  Após o processo, juntarei os dados transformados ao DF padrão

"""
deliveries_df = pd.merge(left=deliveries_df, right=hub_origin_df, how='inner', left_index=True, right_index=True)

# Lendo as 5 primeiras linhas
deliveries_df.head()

In [ ]:
# Removendo a coluna 'origin', ela já foi tratada no código acima
deliveries_df = deliveries_df.drop("origin", axis=1)

# Re-ordenando as colunas e renomeando as novas para o padrão necessário
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]

# Renomeando...
deliveries_df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)

# Lendo as 5 primeiras linhas
deliveries_df.head()

Normalizamos a coluna com uma operação conhecida como `explode` ou explosão, que transforma cada elemento da lista em uma linha. 

In [ ]:
# explodindo a coluna
deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
deliveries_exploded_df.head()

Agora faremos os achatamentos dos resultados da coluna:

In [ ]:
# Usamos funções anônimas para explodir as colunas, e logo renomeamos para o ideal
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)

deliveries_normalized_df.head()

Vamos analisar a diferença dos tamanhos depois que 'explodimos' os dados.

In [ ]:
print(f'Tamanho antigo: {len(deliveries_df)}')
print(f'Novo tamanho: {len(deliveries_exploded_df)}')

Agora vamos combiná-los ao conjunto de dados principal:

In [ ]:
# Antes, vamos remover a coluna 'deliveries' que já foi tratada anteriormente
deliveries_df = deliveries_df.drop("deliveries", axis=1)

# Agora unimos os DataFrames
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)
deliveries_df.head()

O tamanho após a combinação:

In [ ]:
len(deliveries_df)

### **3.3. Estrutura**

Vamos conhecer o conjunto de dados.

In [ ]:
# Quantidade de Linhas e Colunas
deliveries_df.shape

In [ ]:
# Colunas
deliveries_df.columns

In [ ]:
# Index
deliveries_df.index

In [ ]:
# Informações gerais do DF
deliveries_df.info()

### **3.4. Schema**

Analisando colunas e seus respectivos tipos de dados.

In [ ]:
deliveries_df.head(n=5)

* Descrevendo os dados das colunas **categóricas**

In [ ]:
deliveries_df.select_dtypes("object").describe().transpose()

* Descrevendo os dados das colunas **númericas**

In [ ]:
deliveries_df.drop(["name", "region"], axis=1).select_dtypes('int64').describe().transpose()

### **3.5. Dados faltantes** 

Dados faltantes podem ser:

 - Vazios (`""`);
 - Nulos (`None`);
 - Não disponíveis ou aplicaveis (`na`, `NA`, etc.);
 - Não numérico (`nan`, `NaN`, `NAN`, etc).

Antes de passarmos o pente fino nas colunas com dados faltantes, vamos analisar o conjunto inteiro.

In [ ]:
deliveries_df.isnull().any().any()

Não há nenhum dado faltante no conjunto de dados, mas, mesmo assim vamos validar as colunas.

In [ ]:
deliveries_df.isnull().any()

## **4\. Manipulação**

### **4.1. Enriquecimento**

A **geocodificação** é o processo que transforma uma localização descrita por um texto (endereço, nome do local, etc.) em sua respectiva coodernada geográfica *(latitude e longitude)*. A **geocodificação reversa** faz o oposto, transforma uma coordenada geográfica de um local em suas respectivas descrições textuais.

In [ ]:
"""
  Vamos verificar as coordenadas geográficas dos HUBS

"""

hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]]
hub_df = hub_df.drop_duplicates().sort_values(by="region").reset_index(drop=True)
hub_df.head()

O propósito é fazermos uma análise de forma *gratuita*, neste projeto utilizaremos um projeto *open source (ou seja, de código aberto)* chamado de **OpenStreetMap** que mantém um serviço gratuito de geocodificação chamado **Nominatim**, serviço este que apresenta como limitação a quantia de uma única consulta por segundo. Vamos utilizá-lo através do pacote python `geopy` para fazer a operação reversa e enriquecer o nosso `DataFrame` principal.

In [ ]:
# para poder limitar o número de solicitações, o Nominatim requer um valor definido no user_agent, sendo o nome da aplicação
geolocator = Nominatim(user_agent="ebac_geocoder")

# aqui passamos a geolocalização
location = geolocator.reverse("-15.657013854445248, -47.802664728268745")

# analisaremos os valores daquela localização
print(json.dumps(location.raw, indent=2, ensure_ascii=False))

Vamos então aplicar a geocodificação nas coordenadas das três regiões e extrair as informações de **cidade** e **bairro**.

In [ ]:
# definimos o limite de busca a cada 1segundo
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

In [ ]:
"""
  O comando 'reverse' do Nominatim utiliza um padrão para busca, por isso passaremos os
  valores de latitude e longitude para string e concatenaremos com a ", "
"""

hub_df["coordinates"] = hub_df["hub_lat"].astype(str)  + ", " + hub_df["hub_lng"].astype(str) 
hub_df["geodata"] = hub_df["coordinates"].apply(geocoder)
hub_df.head()

In [ ]:
# aqui normalizamos o arquivo transformando os valores (tupla) para linhas
hub_geodata_df = pd.json_normalize(hub_df["geodata"].apply(lambda data: data.raw))
hub_geodata_df.head()

In [ ]:
# geramos um novo DF com as colunas necessárias
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]

# renomeamos as novas colunas
hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)

# uma condicional para tentarmos reduzir os valores nulos
hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(), hub_geodata_df["hub_suburb"], hub_geodata_df["hub_city"])

# removendo uma coluna que não precisamos
hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)
hub_geodata_df.head()

>O `DataFrame` `hub_geodata_df` com as informações de **cidade** e **bairro** é então combinado ao `DataFrame` principal `deliveries_df`, enriquecendo assim o dado.



In [ ]:
hub_df = pd.merge(left=hub_df, right=hub_geodata_df, left_index=True, right_index=True)
hub_df = hub_df[["region", "hub_suburb", "hub_city"]]
hub_df.head()

In [ ]:
# combinando os valores ao DF principal

deliveries_df = pd.merge(left=deliveries_df, right=hub_df, how="inner", on="region")
deliveries_df = deliveries_df[["name", "region", "hub_lng", "hub_lat", "hub_city", "hub_suburb", "vehicle_capacity", "delivery_size", "delivery_lng", "delivery_lat"]]
deliveries_df.head()

### **4.2. Geocodificação reversa da entrega**

Enquanto o **hub** contém apenas **3** geolocalizações distintas, as **entregas** somam o total de **636.149**, o que levaria em torno de 7 dias para serem consultadas no servidor do Nominatim, dada a restrição de uma consulta por segundo. Contudo, para cargas pesadas como esta, o software oferece uma instalação [local](https://nominatim.org/release-docs/latest/admin/Installation/) (na sua própria máquina) que pode ser utilizada sem restrição.

> **Atenção:** Como a instalação do servidor local envolve tecnologias que estão fora do escopo desta análise (como [Docker](https://www.docker.com/)), os dados estão providenciados através deste [link](https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv).

In [ ]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv" -O deliveries-geodata.csv 

In [ ]:
deliveries_geodata_df = pd.read_csv("deliveries-geodata.csv")
deliveries_geodata_df.head()

In [ ]:
# unindo o novo df ao principal
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[["delivery_city", "delivery_suburb"]], how="inner", left_index=True, right_index=True)
deliveries_df.head()

### **4.3. Qualidade**

> Qualidade do dados está relacionado a consistência do `schema`, valores faltantes, etc.

In [ ]:
deliveries_df.info()

 * Notamos que agora contém valores nulos em algumas colunas (del...city e suburb)

In [ ]:
deliveries_df.isna().any()

>**Geocodificação reversa**

% de valores nulos da coluna `city`

In [ ]:
100 * (deliveries_df["delivery_city"].isna().sum() / len(deliveries_df))

% de valores nulos da coluna `suburb`

In [ ]:
100 * (deliveries_df["delivery_suburb"].isna().sum() / len(deliveries_df))

Distribuição de valores da coluna `city`

In [ ]:
prop_df = deliveries_df[["delivery_city"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

Distribuição de valores da coluna `suburb`

In [ ]:
prop_df = deliveries_df[["delivery_suburb"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

## **5\. Visualização**

### **5.1. Mapa de entregas por região**

Vamos utilizar o pacote Python GeopPandas ([link](https://geopandas.org/) da documentação) para visualizar as coordenadas dos **hubs** e das **entregas** no mapa do Distrito Federal, segmentados pela região dos **hubs**. O pacote adiciona funcionalidades geoespaciais ao pacote Python Pandas.

>**Mapa do Distrito Federal**

Vamos fazer o download dos dados do mapa do Distrito Federal do site oficial do IBGE através do seguinte [link](https://www.ibge.gov.br/geociencias/cartas-e-mapas/bases-cartograficas-continuas) para criar o DataFrame `mapa`. Note a coluna `geometry`.

In [ ]:
!wget -q "https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip" -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

In [ ]:
mapa = geopandas.read_file("distrito-federal.shp")
mapa = mapa.loc[[0]]
mapa.head()

>**Mapa dos Hubs**

In [ ]:
hub_df = deliveries_df[["region", "hub_lng", "hub_lat"]].drop_duplicates().reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame(hub_df, geometry=geopandas.points_from_xy(hub_df["hub_lng"], hub_df["hub_lat"]))
geo_hub_df.head()

>**Mapa das Entregas**

In [ ]:
geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df, geometry=geopandas.points_from_xy(deliveries_df["delivery_lng"], deliveries_df["delivery_lat"]))
geo_deliveries_df.head()

>**Visualização**

In [ ]:
# cria o plot vazio
fig, ax = plt.subplots(figsize = (50/2.54, 50/2.54))

# plot mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color="lightgrey")

# plot das entregas
geo_deliveries_df.query("region == 'df-0'").plot(ax=ax, markersize=1, color="red", label="df-0")
geo_deliveries_df.query("region == 'df-1'").plot(ax=ax, markersize=1, color="blue", label="df-1")
geo_deliveries_df.query("region == 'df-2'").plot(ax=ax, markersize=1, color="seagreen", label="df-2")

# plot dos hubs
geo_hub_df.plot(ax=ax, markersize=30, marker="x", color="black", label="hub")

# plot da legenda
plt.title("Entregas no Distrito Federal por Região", fontdict={"fontsize": 16})
lgnd = plt.legend(prop={"size": 15})
for handle in lgnd.legendHandles:
    handle.set_sizes([50])

>**Vamos comparar o resultado com um mapa que mostra a densidade bruta de ocupação no Distrito Federal.** *(Disponível em: [link](https://www.seduh.df.gov.br/mapas/))*

![Mapa de Ocupação Bruta](https://github.com/Tobias-Melo/loggi-AED/blob/main/Densidade_populacional.PNG?raw=true)

>**Insights:**

1. As **entregas** estão corretamente alocadas aos seus respectivos **hubs**;
1. Os **hubs** das regiões 0 e 2 fazem **entregas** em locais distantes do centro e entre si, o que pode gerar um tempo e preço de entrega maior;
1. As entregas do **hub 0** não estão tão concentradas em regiões urbanas quanto aos outros;
1. Há uma distribuição considerável de entregas em zonas rurais no **hub 2**;
1. É possível perceber que as entregas estão diretamente relacionadas com a densidade ocupacional.

### **5.2. Gráfico de entregas por região** 

>**Agregação:**

In [ ]:
data = pd.DataFrame(deliveries_df[['region', 'vehicle_capacity']].value_counts(normalize=True)).reset_index()
data.rename(columns={0: "region_percent"}, inplace=True)
data.head()

>**Visualização:**

In [ ]:
with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=data, x="region", y="region_percent", ci=None, palette="pastel")
  grafico.set(title='Proporção de entregas por região', xlabel='Região', ylabel='Proporção');

>**Insights:**

1. A distribuição das entregas está muito concentrada nos hubs das regiões 1 e 2, mas pouco na região 0. Contudo, a capacidade dos veículos é mesma para todos os hubs, logo os veículos poderiam ser deslocados para as regiões de maior tráfego;
1. No **hub 0** as entregas são distribuídas em viagens longas, por isso há a diferença na proporção de entrega.

## **6. Agradecimentos**

O objetivo principal de todo projeto é identificar as oportunidades daquele negócio/dado, transformá-los em ideias incríveis e o principal... o **aprendizado!**
<br>
Gostaria de agradecer a [EBAC](https://ebaconline.com.br/) e ao professor [André Perez](https://www.linkedin.com/in/andremarcosperez/) pela oportunidade e ensinamentos ao longo do curso, esse é o primeiro projeto de muitos!